In [23]:
from json import loads, dumps, dump
from pathlib import Path
from jcs import canonicalize
from yaml import dump as dumpy
from IPython.display import display, Markdown

out_file = 'canonical-fhir-document.json'
in_file = '/Users/ehaas/Documents/FHIR/davinci-ecdx/CDEX-Signatures/in_files/test_bundle.json'
path = Path() / in_file
path

PosixPath('/Users/ehaas/Documents/FHIR/davinci-ecdx/CDEX-Signatures/in_files/test_bundle.json')

### Get Document

In [24]:
my_dict = loads(path.read_text())
my_dict

{'resourceType': 'Bundle',
 'id': '12521',
 'meta': {'versionId': '1', 'lastUpdated': '2022-02-15T15:53:19.296Z'},
 'identifier': {'system': 'urn:ietf:rfc:3986',
  'value': 'urn:uuid:0c3151bd-1cbf-4d64-b04d-cd9187a4c6e0'},
 'type': 'document',
 'timestamp': '2013-05-28T22:12:21Z',
 'entry': [{'fullUrl': 'urn:uuid:541a72a8-df75-4484-ac89-ac4923f03b81',
   'resource': {'resourceType': 'Observation',
    'referenceRange': [{'text': 'this is my markdown file line 1 \nstill line 1  \nline2\n\nline3\n- bullet 1 \n- bullet 1.\n\n    code "block"\n'}],
    'meta': {'lastUpdated': '2013-05-05T16:13:03Z'},
    'text': {'status': 'additional',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml"> Acute Asthmatic attack. Was wheezing\n\n\n\n                        for days prior to admission. </div>'},
    'status': 'final',
    'code': {'coding': [{'system': 'http://loinc.org', 'code': '46241-6'}],
     'text': 'Reason for admission'},
    'subject': {'reference': 'urn:uuid:d546d85-2487-4ae5-a7

### pop sig, id, meta if there

In [25]:
try:
    my_dict.pop("signature")
except KeyError:
    pass

# http://hl7.org/fhir/canonicalization/json  keep all
# for http://hl7.org/fhir/canonicalization/json#static pop id meta and text elements first
# my_dict.pop("text") recursively

# for http://hl7.org/fhir/canonicalization/json#document pop Bundle.id Bundle.meta elements first
try:
    my_dict.pop("id")
except KeyError:
    pass
try:
    my_dict.pop("meta")
except KeyError:
    pass

my_dict

{'resourceType': 'Bundle',
 'identifier': {'system': 'urn:ietf:rfc:3986',
  'value': 'urn:uuid:0c3151bd-1cbf-4d64-b04d-cd9187a4c6e0'},
 'type': 'document',
 'timestamp': '2013-05-28T22:12:21Z',
 'entry': [{'fullUrl': 'urn:uuid:541a72a8-df75-4484-ac89-ac4923f03b81',
   'resource': {'resourceType': 'Observation',
    'referenceRange': [{'text': 'this is my markdown file line 1 \nstill line 1  \nline2\n\nline3\n- bullet 1 \n- bullet 1.\n\n    code "block"\n'}],
    'meta': {'lastUpdated': '2013-05-05T16:13:03Z'},
    'text': {'status': 'additional',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml"> Acute Asthmatic attack. Was wheezing\n\n\n\n                        for days prior to admission. </div>'},
    'status': 'final',
    'code': {'coding': [{'system': 'http://loinc.org', 'code': '46241-6'}],
     'text': 'Reason for admission'},
    'subject': {'reference': 'urn:uuid:d546d85-2487-4ae5-a7b2-f7fa79811120',
     'display': 'Eve Everywoman'},
    'encounter': {'reference': 'urn

### sort and remove white space

In [26]:
display(Markdown(my_dict['entry'][0]['resource']['referenceRange'][0]['text']))

this is my markdown file line 1 
still line 1  
line2

line3
- bullet 1 
- bullet 1.

    code "block"


In [27]:
canon_json = canonicalize(my_dict)
canon_json

b'{"entry":[{"fullUrl":"urn:uuid:541a72a8-df75-4484-ac89-ac4923f03b81","resource":{"code":{"coding":[{"code":"46241-6","system":"http://loinc.org"}],"text":"Reason for admission"},"encounter":{"reference":"urn:uuid:a16719d5-5267-4f69-913a-f29e4b800a14"},"meta":{"lastUpdated":"2013-05-05T16:13:03Z"},"referenceRange":[{"text":"this is my markdown file line 1 \\nstill line 1  \\nline2\\n\\nline3\\n- bullet 1 \\n- bullet 1.\\n\\n    code \\"block\\"\\n"}],"resourceType":"Observation","status":"final","subject":{"display":"Eve Everywoman","reference":"urn:uuid:d546d85-2487-4ae5-a7b2-f7fa79811120"},"text":{"div":"<div xmlns=\\"http://www.w3.org/1999/xhtml\\"> Acute Asthmatic attack. Was wheezing\\n\\n\\n\\n                        for days prior to admission. </div>","status":"additional"}}}],"identifier":{"system":"urn:ietf:rfc:3986","value":"urn:uuid:0c3151bd-1cbf-4d64-b04d-cd9187a4c6e0"},"resourceType":"Bundle","timestamp":"2013-05-28T22:12:21Z","type":"document"}'

### Write to file

In [28]:

out_path = Path() / out_file
out_path.write_bytes(canon_json)

955

In [29]:
json_string = canon_json.decode('utf-8')
data = loads(json_string)
data


{'entry': [{'fullUrl': 'urn:uuid:541a72a8-df75-4484-ac89-ac4923f03b81',
   'resource': {'code': {'coding': [{'code': '46241-6',
       'system': 'http://loinc.org'}],
     'text': 'Reason for admission'},
    'encounter': {'reference': 'urn:uuid:a16719d5-5267-4f69-913a-f29e4b800a14'},
    'meta': {'lastUpdated': '2013-05-05T16:13:03Z'},
    'referenceRange': [{'text': 'this is my markdown file line 1 \nstill line 1  \nline2\n\nline3\n- bullet 1 \n- bullet 1.\n\n    code "block"\n'}],
    'resourceType': 'Observation',
    'status': 'final',
    'subject': {'display': 'Eve Everywoman',
     'reference': 'urn:uuid:d546d85-2487-4ae5-a7b2-f7fa79811120'},
    'text': {'div': '<div xmlns="http://www.w3.org/1999/xhtml"> Acute Asthmatic attack. Was wheezing\n\n\n\n                        for days prior to admission. </div>',
     'status': 'additional'}}}],
 'identifier': {'system': 'urn:ietf:rfc:3986',
  'value': 'urn:uuid:0c3151bd-1cbf-4d64-b04d-cd9187a4c6e0'},
 'resourceType': 'Bundle',
 't

In [33]:
print(dumps(data['entry'][0]['resource']['text']['div'],indent=2))


"<div xmlns=\"http://www.w3.org/1999/xhtml\"> Acute Asthmatic attack. Was wheezing\n\n\n\n                        for days prior to admission. </div>"


In [31]:
display(Markdown(data['entry'][0]['resource']['referenceRange'][0]['text']))

this is my markdown file line 1 
still line 1  
line2

line3
- bullet 1 
- bullet 1.

    code "block"
